In [11]:
import socket
import struct
import sys

In [ ]:
from stanag4586edav1.message_wrapper import *

#### Open socket

In [12]:
sock_tx = ""
sock_rx = ""
mc_tx = ""
mc_rx = ""

In [14]:
def setup_networking(rx_timeout = 2.0):
    
    global sock_tx, sock_rx, mc_tx, mc_rx
    
    ttl = struct.pack('b',1)
    
    mc_tx = ('224.10.10.10', 4586)
    sock_tx = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock_tx.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

    mc_rx = '224.10.10.10'
    sock_rx = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock_rx.settimeout(rx_timeout)
    group = socket.inet_aton(mc_rx)
    mreq = struct.pack('4sL', group, socket.INADDR_ANY)
    sock_rx.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)
    sock_rx.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock_rx.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)

    sock_rx.bind(('', 4587))


#### Wait for response

In [10]:
def do_rx(num_msgs_to_expect = 1):
    print("Begin waiting for [{}] messages".format(num_msgs_to_expect))
    data = ""
    
    for i in range(0, num_msgs_to_expect):
        try:
            data, sender_address = sock_rx.recvfrom(1000)
            wrapper = MessageWrapper(data)
            print("Got message [{}]".format(wrapper.message_type))

        except socket.timeout:
            print("Timed out waiting for msg#{}, will not wait for any other messages.".format(i))
            break
    
    print("End waiting for [{}] messages".format(num_msgs_to_expect))
    
    return data